All required imports

In [1]:
import numpy as np
import matplotlib as plt
import sklearn
import os
import pandas as pd
#import keras

This loads data into pandas dataframes

In [2]:
try:
    root
except:
    root = os.getcwd()

data_path = os.path.join(root, "../skin-cancer-mnist-ham10000")

# the associated meta data
metadata = pd.read_csv(os.path.join(data_path, "HAM10000_metadata.csv"))

# the pixels RGB
X_pixels = pd.read_csv(os.path.join(data_path, "hmnist_28_28_RGB.csv"))

In [3]:
print(metadata.shape)
metadata.head()

(10015, 7)


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [4]:
print(X_pixels.shape)
X_pixels.drop(['label'], axis=1,inplace = True)
X_pixels.head()

(10015, 2353)


,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,192,153,193,195,155,192,197,154,185,202,...,134,173,124,138,183,147,166,185,154,177
1,25,14,30,68,48,75,123,93,126,158,...,82,60,39,55,25,14,28,25,14,27
2,192,138,153,200,145,163,201,142,160,206,...,149,167,129,143,159,124,142,136,104,117
3,38,19,30,95,59,72,143,103,119,171,...,73,44,26,36,25,12,17,25,12,15
4,158,113,139,194,144,174,215,162,191,225,...,201,209,166,185,172,135,149,109,78,92


Reformat metadata into X_metadata, categorical variables turned into one hot encoding, non-features removed.
Also make variable y into the classifications

In [5]:
X_metadata = metadata.loc[:, ['dx_type', 'age', 'sex', 'localization']]
X_metadata = pd.get_dummies(data=X_metadata, columns=['dx_type', 'sex', 'localization'])
X_metadata.head()

,age,dx_type_confocal,dx_type_consensus,dx_type_follow_up,dx_type_histo,sex_female,sex_male,sex_unknown,localization_abdomen,localization_acral,...,localization_face,localization_foot,localization_genital,localization_hand,localization_lower extremity,localization_neck,localization_scalp,localization_trunk,localization_unknown,localization_upper extremity
0,80.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,80.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,80.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,80.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,75.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y = metadata.loc[:, 'dx']
y = pd.get_dummies(data=y, columns=['dx'])
y.head()

,akiec,bcc,bkl,df,mel,nv,vasc
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0


Simply make X be the concatenation of X_pixels and X_metadata

In [7]:
X = pd.concat([X_metadata, X_pixels], axis = 1)
X.head()

,age,dx_type_confocal,dx_type_consensus,dx_type_follow_up,dx_type_histo,sex_female,sex_male,sex_unknown,localization_abdomen,localization_acral,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,80.0,0,0,0,1,0,1,0,0,0,...,134,173,124,138,183,147,166,185,154,177
1,80.0,0,0,0,1,0,1,0,0,0,...,82,60,39,55,25,14,28,25,14,27
2,80.0,0,0,0,1,0,1,0,0,0,...,149,167,129,143,159,124,142,136,104,117
3,80.0,0,0,0,1,0,1,0,0,0,...,73,44,26,36,25,12,17,25,12,15
4,75.0,0,0,0,1,0,1,0,0,0,...,201,209,166,185,172,135,149,109,78,92


This is going to be all of my analysis using CNN from keras: Samuel Stentz.

I am also going to be using some slight data augmentation to help make the classifier more robust

https://medium.com/datadriveninvestor/keras-imagedatagenerator-methods-an-easy-guide-550ecd3c0a92

This is the article some of this code is from which we should cite

In [8]:
# move all images to one directory
import shutil
im1 = os.path.join(data_path, "HAM10000_images_part_1")
im2 = os.path.join(data_path, "HAM10000_images_part_2")

i = 0
for im in os.listdir(im2):
    i += 1
    print(f"im {i} moved")
    shutil.move(os.path.join(im2,im), im1)

im 1 moved
im 2 moved
im 3 moved
im 4 moved
im 5 moved
im 6 moved
im 7 moved
im 8 moved
im 9 moved
im 10 moved
im 11 moved
im 12 moved
im 13 moved
im 14 moved
im 15 moved
im 16 moved
im 17 moved
im 18 moved
im 19 moved
im 20 moved
im 21 moved
im 22 moved
im 23 moved
im 24 moved
im 25 moved
im 26 moved
im 27 moved
im 28 moved
im 29 moved
im 30 moved
im 31 moved
im 32 moved
im 33 moved
im 34 moved
im 35 moved
im 36 moved
im 37 moved
im 38 moved
im 39 moved
im 40 moved
im 41 moved
im 42 moved
im 43 moved
im 44 moved
im 45 moved
im 46 moved
im 47 moved
im 48 moved
im 49 moved
im 50 moved
im 51 moved
im 52 moved
im 53 moved
im 54 moved
im 55 moved
im 56 moved
im 57 moved
im 58 moved
im 59 moved
im 60 moved
im 61 moved
im 62 moved
im 63 moved
im 64 moved
im 65 moved
im 66 moved
im 67 moved
im 68 moved
im 69 moved
im 70 moved
im 71 moved
im 72 moved
im 73 moved
im 74 moved
im 75 moved
im 76 moved
im 77 moved
im 78 moved
im 79 moved
im 80 moved
im 81 moved
im 82 moved
im 83 moved
im 84 moved
i

im 704 moved
im 705 moved
im 706 moved
im 707 moved
im 708 moved
im 709 moved
im 710 moved
im 711 moved
im 712 moved
im 713 moved
im 714 moved
im 715 moved
im 716 moved
im 717 moved
im 718 moved
im 719 moved
im 720 moved
im 721 moved
im 722 moved
im 723 moved
im 724 moved
im 725 moved
im 726 moved
im 727 moved
im 728 moved
im 729 moved
im 730 moved
im 731 moved
im 732 moved
im 733 moved
im 734 moved
im 735 moved
im 736 moved
im 737 moved
im 738 moved
im 739 moved
im 740 moved
im 741 moved
im 742 moved
im 743 moved
im 744 moved
im 745 moved
im 746 moved
im 747 moved
im 748 moved
im 749 moved
im 750 moved
im 751 moved
im 752 moved
im 753 moved
im 754 moved
im 755 moved
im 756 moved
im 757 moved
im 758 moved
im 759 moved
im 760 moved
im 761 moved
im 762 moved
im 763 moved
im 764 moved
im 765 moved
im 766 moved
im 767 moved
im 768 moved
im 769 moved
im 770 moved
im 771 moved
im 772 moved
im 773 moved
im 774 moved
im 775 moved
im 776 moved
im 777 moved
im 778 moved
im 779 moved
im 780 moved

im 1312 moved
im 1313 moved
im 1314 moved
im 1315 moved
im 1316 moved
im 1317 moved
im 1318 moved
im 1319 moved
im 1320 moved
im 1321 moved
im 1322 moved
im 1323 moved
im 1324 moved
im 1325 moved
im 1326 moved
im 1327 moved
im 1328 moved
im 1329 moved
im 1330 moved
im 1331 moved
im 1332 moved
im 1333 moved
im 1334 moved
im 1335 moved
im 1336 moved
im 1337 moved
im 1338 moved
im 1339 moved
im 1340 moved
im 1341 moved
im 1342 moved
im 1343 moved
im 1344 moved
im 1345 moved
im 1346 moved
im 1347 moved
im 1348 moved
im 1349 moved
im 1350 moved
im 1351 moved
im 1352 moved
im 1353 moved
im 1354 moved
im 1355 moved
im 1356 moved
im 1357 moved
im 1358 moved
im 1359 moved
im 1360 moved
im 1361 moved
im 1362 moved
im 1363 moved
im 1364 moved
im 1365 moved
im 1366 moved
im 1367 moved
im 1368 moved
im 1369 moved
im 1370 moved
im 1371 moved
im 1372 moved
im 1373 moved
im 1374 moved
im 1375 moved
im 1376 moved
im 1377 moved
im 1378 moved
im 1379 moved
im 1380 moved
im 1381 moved
im 1382 moved
im 138

im 1911 moved
im 1912 moved
im 1913 moved
im 1914 moved
im 1915 moved
im 1916 moved
im 1917 moved
im 1918 moved
im 1919 moved
im 1920 moved
im 1921 moved
im 1922 moved
im 1923 moved
im 1924 moved
im 1925 moved
im 1926 moved
im 1927 moved
im 1928 moved
im 1929 moved
im 1930 moved
im 1931 moved
im 1932 moved
im 1933 moved
im 1934 moved
im 1935 moved
im 1936 moved
im 1937 moved
im 1938 moved
im 1939 moved
im 1940 moved
im 1941 moved
im 1942 moved
im 1943 moved
im 1944 moved
im 1945 moved
im 1946 moved
im 1947 moved
im 1948 moved
im 1949 moved
im 1950 moved
im 1951 moved
im 1952 moved
im 1953 moved
im 1954 moved
im 1955 moved
im 1956 moved
im 1957 moved
im 1958 moved
im 1959 moved
im 1960 moved
im 1961 moved
im 1962 moved
im 1963 moved
im 1964 moved
im 1965 moved
im 1966 moved
im 1967 moved
im 1968 moved
im 1969 moved
im 1970 moved
im 1971 moved
im 1972 moved
im 1973 moved
im 1974 moved
im 1975 moved
im 1976 moved
im 1977 moved
im 1978 moved
im 1979 moved
im 1980 moved
im 1981 moved
im 198

im 2564 moved
im 2565 moved
im 2566 moved
im 2567 moved
im 2568 moved
im 2569 moved
im 2570 moved
im 2571 moved
im 2572 moved
im 2573 moved
im 2574 moved
im 2575 moved
im 2576 moved
im 2577 moved
im 2578 moved
im 2579 moved
im 2580 moved
im 2581 moved
im 2582 moved
im 2583 moved
im 2584 moved
im 2585 moved
im 2586 moved
im 2587 moved
im 2588 moved
im 2589 moved
im 2590 moved
im 2591 moved
im 2592 moved
im 2593 moved
im 2594 moved
im 2595 moved
im 2596 moved
im 2597 moved
im 2598 moved
im 2599 moved
im 2600 moved
im 2601 moved
im 2602 moved
im 2603 moved
im 2604 moved
im 2605 moved
im 2606 moved
im 2607 moved
im 2608 moved
im 2609 moved
im 2610 moved
im 2611 moved
im 2612 moved
im 2613 moved
im 2614 moved
im 2615 moved
im 2616 moved
im 2617 moved
im 2618 moved
im 2619 moved
im 2620 moved
im 2621 moved
im 2622 moved
im 2623 moved
im 2624 moved
im 2625 moved
im 2626 moved
im 2627 moved
im 2628 moved
im 2629 moved
im 2630 moved
im 2631 moved
im 2632 moved
im 2633 moved
im 2634 moved
im 263

im 3193 moved
im 3194 moved
im 3195 moved
im 3196 moved
im 3197 moved
im 3198 moved
im 3199 moved
im 3200 moved
im 3201 moved
im 3202 moved
im 3203 moved
im 3204 moved
im 3205 moved
im 3206 moved
im 3207 moved
im 3208 moved
im 3209 moved
im 3210 moved
im 3211 moved
im 3212 moved
im 3213 moved
im 3214 moved
im 3215 moved
im 3216 moved
im 3217 moved
im 3218 moved
im 3219 moved
im 3220 moved
im 3221 moved
im 3222 moved
im 3223 moved
im 3224 moved
im 3225 moved
im 3226 moved
im 3227 moved
im 3228 moved
im 3229 moved
im 3230 moved
im 3231 moved
im 3232 moved
im 3233 moved
im 3234 moved
im 3235 moved
im 3236 moved
im 3237 moved
im 3238 moved
im 3239 moved
im 3240 moved
im 3241 moved
im 3242 moved
im 3243 moved
im 3244 moved
im 3245 moved
im 3246 moved
im 3247 moved
im 3248 moved
im 3249 moved
im 3250 moved
im 3251 moved
im 3252 moved
im 3253 moved
im 3254 moved
im 3255 moved
im 3256 moved
im 3257 moved
im 3258 moved
im 3259 moved
im 3260 moved
im 3261 moved
im 3262 moved
im 3263 moved
im 326

im 3792 moved
im 3793 moved
im 3794 moved
im 3795 moved
im 3796 moved
im 3797 moved
im 3798 moved
im 3799 moved
im 3800 moved
im 3801 moved
im 3802 moved
im 3803 moved
im 3804 moved
im 3805 moved
im 3806 moved
im 3807 moved
im 3808 moved
im 3809 moved
im 3810 moved
im 3811 moved
im 3812 moved
im 3813 moved
im 3814 moved
im 3815 moved
im 3816 moved
im 3817 moved
im 3818 moved
im 3819 moved
im 3820 moved
im 3821 moved
im 3822 moved
im 3823 moved
im 3824 moved
im 3825 moved
im 3826 moved
im 3827 moved
im 3828 moved
im 3829 moved
im 3830 moved
im 3831 moved
im 3832 moved
im 3833 moved
im 3834 moved
im 3835 moved
im 3836 moved
im 3837 moved
im 3838 moved
im 3839 moved
im 3840 moved
im 3841 moved
im 3842 moved
im 3843 moved
im 3844 moved
im 3845 moved
im 3846 moved
im 3847 moved
im 3848 moved
im 3849 moved
im 3850 moved
im 3851 moved
im 3852 moved
im 3853 moved
im 3854 moved
im 3855 moved
im 3856 moved
im 3857 moved
im 3858 moved
im 3859 moved
im 3860 moved
im 3861 moved
im 3862 moved
im 386

im 4443 moved
im 4444 moved
im 4445 moved
im 4446 moved
im 4447 moved
im 4448 moved
im 4449 moved
im 4450 moved
im 4451 moved
im 4452 moved
im 4453 moved
im 4454 moved
im 4455 moved
im 4456 moved
im 4457 moved
im 4458 moved
im 4459 moved
im 4460 moved
im 4461 moved
im 4462 moved
im 4463 moved
im 4464 moved
im 4465 moved
im 4466 moved
im 4467 moved
im 4468 moved
im 4469 moved
im 4470 moved
im 4471 moved
im 4472 moved
im 4473 moved
im 4474 moved
im 4475 moved
im 4476 moved
im 4477 moved
im 4478 moved
im 4479 moved
im 4480 moved
im 4481 moved
im 4482 moved
im 4483 moved
im 4484 moved
im 4485 moved
im 4486 moved
im 4487 moved
im 4488 moved
im 4489 moved
im 4490 moved
im 4491 moved
im 4492 moved
im 4493 moved
im 4494 moved
im 4495 moved
im 4496 moved
im 4497 moved
im 4498 moved
im 4499 moved
im 4500 moved
im 4501 moved
im 4502 moved
im 4503 moved
im 4504 moved
im 4505 moved
im 4506 moved
im 4507 moved
im 4508 moved
im 4509 moved
im 4510 moved
im 4511 moved
im 4512 moved
im 4513 moved
im 451

In [9]:
dataset = metadata.loc[:,["dx", "image_id"]]
dataset.columns = ["label", "filename"]

def to_jpg(x):
    return f"{x}.jpg"

dataset['filename'] = dataset['filename'].apply(lambda x: f"{x}.jpg")
dataset.head()

from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state = 42)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state = 42)

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import ModelCheckpoint

# where all the data comes from
train_data_dir = im1

# dimensions of our images.
img_width, img_height = 150, 150

nb_train_samples = 6409
nb_validation_samples = 1603
nb_test_samples = 2003
epochs = 50
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#############################################################################################################
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
##############################################################################################################
"""shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split"""

# this is the augmentation configuration we will use for training (Nothing!!!)
train_datagen = ImageDataGenerator(
    rescale=1./255
)
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# all the generators for train, val, and test
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_dataset,
    directory = train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical')

validation_generator = train_datagen.flow_from_dataframe(
    dataframe = val_dataset,
    directory = train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_dataset,
    directory = train_data_dir,
    target_size= (img_height, img_width),
    batch_size= batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical',
    shuffle = False)

Using TensorFlow backend.
C:\Users\prajw\Anaconda3\envs\mlproj\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\prajw\Anaconda3\envs\mlproj\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\prajw\Anaconda3\envs\mlproj\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\prajw\Anaconda3\envs\mlproj\lib\sit






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Found 6409 validated image filenames belonging to 7 classes.
Found 1603 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


This is the training of the model

In [ ]:
# save the model with best validation accuracy
mcp = ModelCheckpoint("best_validation.hdf5", monitor="val_accuracy", save_best_only=True, save_weights_only=False)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size + 1,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size + 1,
    callbacks=[mcp])

model.save_weights('first_try.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
190/201 [===========================>..] - ETA: 5s - loss: 1.1498 - acc: 0.6533

In [ ]:
# load model and evaluate on test set
from tensorflow.keras.models import load_model

model.load_weights('best_validation.hdf5')

test_generator.reset()
y_pred = model.predict_generator(test_generator,steps= nb_test_samples // batch_size + 1)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
label_map = test_generator.class_indices
reverse = dict()
for l in label_map:
    reverse[label_map[l]] = l

names = []
for number in reverse:
    names.append(reverse[number])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
labels = test_generator.classes

print('Confusion Matrix')
print(confusion_matrix(labels, y_pred))
print('\n\n\nClassification Report')
print(classification_report(labels, y_pred, target_names=names))
num_right = np.sum(labels == y_pred)
print(f"\nAccuracy {num_right / y_pred.shape[0]}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import seaborn as sns
import math

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib as mpl

mpl.style.use('seaborn')

conf_arr = confusion_matrix(labels, y_pred)

s = conf_arr.sum(axis = 1)

conf_arr = (conf_arr * 100.0) / s[:,None]

conf_arr = np.nan_to_num(conf_arr)

df_cm = pd.DataFrame(conf_arr, 
  index = names,
  columns = names)

fig = plt.figure()

plt.clf()

ax = fig.add_subplot(111)
ax.set_aspect(1)

cmap = sns.cubehelix_palette(light=1, as_cmap=True)

res = sn.heatmap(df_cm, annot=True, vmin=0.0, vmax=100.0, fmt='.2f', cmap=cmap)

res.invert_yaxis()

plt.yticks(np.arange(len(names)) + .5, names,va='center')

plt.title('Percent Classified')
plt.xlabel("Prediction")
plt.ylabel("Class")

plt.savefig('confusion_matrix_cnn.png', dpi=100, bbox_inches='tight' )
plt.show()
plt.close()

Try a different architechture

In [ ]:
import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
mobile = tensorflow.keras.applications.mobilenet.MobileNet()
# CREATE THE MODEL ARCHITECTURE

# Exclude the last 5 layers of the above model.
# This will include all layers up to and including global_average_pooling2d_1
x = mobile.layers[-6].output

# Create a new dense layer for predictions
# 7 corresponds to the number of classes
x = Dropout(0.25)(x)

predictions = Dense(7, activation='softmax')(x)

# inputs=mobile.input selects the input layer, outputs=predictions refers to the
# dense layer we created above.

model = Model(inputs=mobile.input, outputs=predictions)

model.summary()

for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

model.compile(Adam(lr=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

In [ ]:
# where all the data comes from
train_data_dir = im1

# dimensions of our images.
img_width, img_height = 224, 224

nb_train_samples = 6409
nb_validation_samples = 1603
nb_test_samples = 2003
epochs = 50
batch_size = 32

# this is the augmentation configuration we will use for training (Nothing!!!)
train_datagen = ImageDataGenerator(
    rescale=1./255
)
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# all the generators for train, val, and test
train_generator = train_datagen.flow_from_dataframe(
    preprocessing_function= \
    tensorflow.keras.applications.mobilenet.preprocess_input,
    dataframe = train_dataset,
    directory = train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical')

validation_generator = train_datagen.flow_from_dataframe(
    preprocessing_function= \
    tensorflow.keras.applications.mobilenet.preprocess_input,
    dataframe = val_dataset,
    directory = train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    preprocessing_function= \
    tensorflow.keras.applications.mobilenet.preprocess_input,
    dataframe = test_dataset,
    directory = train_data_dir,
    target_size= (img_height, img_width),
    batch_size= batch_size,
    x_col = "filename",
    y_col = "label",
    class_mode='categorical',
    shuffle = False)

In [ ]:
mcp = ModelCheckpoint("best_validation_mobilenet.hdf5", monitor="val_categorical_accuracy", save_best_only=True, save_weights_only=False)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size + 1,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size + 1,
    callbacks=[mcp])

In [ ]:
https://www.kaggle.com/vbookshelf